# Estimate the number of people in each parish who were infected by the start of the first testing phase


# Background
- Even when specificity is high, a high proportion of positive results may be false positives.
- eg. if sensitivity=70% and specificity=98% and and if we assume an actual infection prevalence of around 10%, this implies that over 20% of all positive results are False positives. (=90 x 0.02/(10 x 0.7 + 90 x 0.02)).
- Therefore, we must account for the sensitivity and specificity in our estimates of infection rate to avoid over-estimating.



Using Bayes method, the probability of an actual infection (D) given a positive test result (T) is given by:<br>

\begin{equation*}
P(D|T) = P(T|D)P(D) / P(T) \\
P(D|T) = \frac{se \times P(D) }{ se \times P(D) + (1-sp) \times P(D') }
\end{equation*}

The sensitivity (se) and specificity (sp) are attributes of the tests. The prior probability of being infected P(D) and its reciprocal P(D') have to be either known in advance or estimated from the data. A common way to estimate this from the data is using maximum likelihood estimation:
https://academic.oup.com/aje/article-abstract/107/1/71/104051?redirectedFrom=fulltext


\begin{equation*}
P(D) = \frac{(\frac{n^T}{N} - (1-sp))}{(se+sp-1)} 
\end{equation*}

where $n^T$ is the number of positive results and N is the total number of people tested

In [64]:
import pandas as pd
from collections import Counter
import datetime

In [65]:
serology=pd.read_csv('../data/private/serology_clean.csv')

/Users/doorleyr/anaconda/envs/shapely/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (4,7,11,16,18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [66]:
print(len(serology))
serology=serology.loc[~serology['data_test_1'].isnull()]
print(len(serology))

73265
70626


In [ ]:
serology_accuracy=json.load(open('data/private/serology_accuracy.json'))

In [67]:
se=serology_accuracy['se']
sp=serology_accuracy['sp']

Combine AlV and Escaldes, fix parish names

In [68]:
from collections import Counter
Counter(serology['parish'])

Counter({'escaldes_engordany': 28924,
         'andorra_la_vella': 4542,
         'sant_julia': 8632,
         'la_massana': 8843,
         'encamp': 10846,
         'canillo': 4495,
         'ordino': 4214,
         nan: 130})

In [69]:
def new_parish_name(parish):
    if isinstance(parish, str):
        if parish in ['andorra_la_vella', 'escaldes_engordany']:
            return 'Andorra la Vella & Escaldes-Engordany'
        elif parish=='sant_julia':
            return 'Sant Juliá de Lòria'
        else:
            return parish.replace('_', ' ').title()
    else:
        return float('nan')
    
serology['new_parish']=serology.apply(lambda row: new_parish_name(row['parish']), axis=1)

### Descriptive stats of dataset

In [70]:
n_by_parish_status=pd.crosstab(serology['new_parish'], serology['temporer'])
n_by_parish_status.to_csv('../outputs/n_by_parish_status.csv')
n_by_parish_status

temporer,0.0,1.0
new_parish,,
Andorra la Vella & Escaldes-Engordany,32925,436
Canillo,3418,1071
Encamp,10115,715
La Massana,8460,369
Ordino,4131,74
Sant Juliá de Lòria,8578,34


# Estimate the percentage of people who were infected before the first test

get the percentage of people who tested positive for any antibody in each period

In [71]:
serology['naive_any_antibody_test_1']=((serology['igm_1']=='positiu')|(serology['igg_1']=='positiu'))
serology['naive_any_antibody_test_2']=((serology['igm_2']=='positiu')|(serology['igg_2']=='positiu'))
print(sum(serology['naive_any_antibody_test_1']))
print(sum(serology['naive_any_antibody_test_2']))

6812
5675


In [72]:
prop_period_1=sum(serology['naive_any_antibody_test_1'])/len(serology)
print(prop_period_1)

prop_period_2=sum(serology['naive_any_antibody_test_2'])/len(serology)
print(prop_period_2)

0.0964517316568969
0.08035284456149293


confirm the conditions of the max likelihood formula are satisfied

In [73]:
assert (1-sp) < prop_period_1
assert prop_period_1 < se

finally, estimate the percentage of people infected before period 1 (based on test phase 1 results only)

In [74]:
prior_prob_period_1 = (prop_period_1 - (1-sp))/(se+sp-1)
print(prior_prob_period_1)

0.11079961109695201


In [75]:
print('Reporting factor: {}'.format(prior_prob_period_1*len(serology)/745))

Reporting factor: 10.50380313199105


In [76]:
def get_posterior(row, prior, L_given_T1, L_given_notT1):
    any_ig_1= row['naive_any_antibody_test_1']
    if any_ig_1:
        return prior*L_given_T1
    else:
        return prior*L_given_notT1    

## Estimate each person's probability based on test 1 result only

Likelihood of D1 given 1 test result

In [78]:
# P(D|T)=P(T|D)P(D)/P(T)
L_D1_given_T1 = se/prop_period_1
print(L_D1_given_T1)
# P(D|T')=P(T'|D)P(D)/P(T')
L_D1_given_notT1 = (1-se)/(1-prop_period_1)
print(L_D1_given_notT1)

7.36119495008808
0.320956843325916


In [80]:
serology['P_D1_test_1_only']=serology.apply(lambda row: get_posterior(row, prior_prob_period_1, L_D1_given_T1, L_D1_given_notT1), axis=1)

In [ ]:
test1_by_parish=serology.groupby('new_parish')['P_D1_test_1_only'].mean().round(2).reset_index()
test1_by_status=serology.groupby( 'temporer')['P_D1_test_1_only'].agg(['mean', 'size']).round(2)

## Test significance of status and parish on infection rate

In [84]:
serology['P_not_D1_test_1_only']=1-serology['P_D1_test_1_only']
infection_by_status=pd.DataFrame(serology.groupby('temporer')[['P_D1_test_1_only', 'P_not_D1_test_1_only']].sum())
infection_by_parish=pd.DataFrame(serology.groupby('parish')[['P_D1_test_1_only', 'P_not_D1_test_1_only']].sum())
infection_by_parish

,P_D1_test_1_only,P_not_D1_test_1_only
parish,,
andorra_la_vella,418.940483,4123.059517
canillo,498.394861,3996.605139
encamp,1101.015322,9744.984678
escaldes_engordany,2984.191705,25939.808295
la_massana,1222.458593,7620.541407
ordino,463.440188,3750.559812
sant_julia,1116.668023,7515.331977


In [85]:
from scipy.stats import chi2_contingency, chi2
stat, p, dof, expected = chi2_contingency(infection_by_status)
print('dof=%d' % dof)
print(p)
print('P: {}'.format(p))
print(expected)
# interpret test-statistic
prob = 0.99
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

dof=1
1.4231596764344866e-06
P: 1.4231596764344866e-06
[[ 7479.04613822 60261.95386179]
 [  299.64321783  2414.35678217]]
probability=0.990, critical=6.635, stat=23.249
Dependent (reject H0)
significance=0.010, p=0.000
Dependent (reject H0)


In [86]:
stat, p, dof, expected = chi2_contingency(infection_by_parish)
print('dof=%d' % dof)
print(p)
print('P: {}'.format(p))
print(expected)
# interpret test-statistic
prob = 0.99
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

dof=6
8.468784059359277e-28
P: 8.468784059359277e-28
[[  502.87684223  4039.12315777]
 [  497.67314087  3997.32685913]
 [ 1200.83712699  9645.16287301]
 [ 3202.37996138 25721.62003863]
 [  979.07087535  7863.92912466]
 [  466.56164975  3747.43835025]
 [  955.70957774  7676.29042226]]
probability=0.990, critical=16.812, stat=140.347
Dependent (reject H0)
significance=0.010, p=0.000
Dependent (reject H0)


Geographic correlation of temp infections to other infections

In [87]:
test1_by_parish_status.loc[test1_by_parish_status['temporer']==0, 'P_D1_test_1_only'].values

array([0.1 , 0.11, 0.1 , 0.14, 0.11, 0.13])

In [88]:
from scipy.stats import pearsonr
pearsonr(test1_by_parish_status.loc[test1_by_parish_status['temporer']==0, 'P_D1_test_1_only'].values,
       test1_by_parish_status.loc[test1_by_parish_status['temporer']==1, 'P_D1_test_1_only'].values)

(0.5034538273357869, 0.3086234115399094)